In [683]:
import pandas as pd
import os
import numpy as np
import re
import datetime

In [684]:
pd.set_option('display.max_columns', 500)

In [685]:
ref_df = pd.read_csv('which_race.csv', header=0)
csv_list = os.listdir('clean_frames')
counter = 34

In [686]:
d2 = {}
d3 = {}
for race_id in range(counter):
    d2['race_df_'+str(race_id)]=pd.read_csv('clean_frames/race_df_'+str(race_id)+'.csv')
    d3['pit_stop_df_'+str(race_id)]=pd.read_csv('clean_frames/pit_stop_df_'+str(race_id)+'.csv')

In [687]:
# for race_id in range(counter):
#     race_df = d2['race_df_'+str(race_id)]
# #     if race_df.index.get_level_values('LAP').max()!=race_df.index.get_level_values('LAP').nunique():
#     if race_df['LAP'].max()!=race_df['LAP'].nunique():
# #         print(counter, "actual laps:",race_df.index.get_level_values('LAP').nunique(), "index laps" ,race_df.index.get_level_values('LAP').max()==race_df.index.get_level_values('LAP').nunique())
#         print(race_id, "actual laps:",race_df['LAP'].nunique(), "index laps" ,race_df['LAP'].max())
#         if race_df['LAP'].max()>race_df['LAP'].nunique():
#             print('at least it makes sense')

In [688]:
# new_index = [x for x in range(1,race_df.index.get_level_values('LAP').nunique()+1)]
# #race_df.index.get_level_values('LAP').nunique()

In [689]:
# for i in d2.keys():
#     print(i,'is',d2[i].shape)

In [690]:
def distances(x):
    return x.values[:,None]-x.values

In [691]:
race_df = d2['race_df_4']
# dfone = pd.concat(d2.values())
pit_stop_df = d3['pit_stop_df_4']

In [692]:
race_df['TIME'] = race_df['TIME'].apply(lambda x: int(x.split(':')[0])*60 + float(x.split(':')[1]))

In [693]:
race_df['cumtime']=race_df[['TIME','NO']].groupby('NO').cumsum(skipna=True)

In [694]:
# pit_stop_df.columns

In [695]:
race_df.drop(columns = ['Unnamed: 0'], inplace=True)

In [696]:
pit_stop_df.drop(columns = ['Unnamed: 0','Unnamed: 0.1','Unnamed: 6'], inplace=True)

In [697]:
last_lap = max(race_df['LAP'])

In [698]:
race_df['progress'] =  race_df['LAP']/last_lap

In [699]:
# race_df['temp_lap'] =  race_df['LAP']

In [700]:
race_df.set_index(['NO','LAP'], inplace=True)

In [701]:
race_df = race_df[~race_df.index.duplicated(keep='first')]

In [702]:
new_index = [x for x in range(1,race_df.index.get_level_values('LAP').nunique()+1)]

reindex_d = dict(zip(list(race_df.index.get_level_values('LAP').unique()), new_index))

race_df.rename(index=reindex_d,level=1,inplace=True)

In [703]:
race_df['progress'] = race_df.index.get_level_values('LAP')/(race_df.index.get_level_values('LAP').nunique())

In [704]:
d_gaps={}
for i in range(0,last_lap):
    lap = i+1
#     gaps_df = pd.DataFrame(distances(race_df['TIME'][race_df['LAP']==lap]),columns=list(race_df['NO'][race_df['LAP']==lap ].values),index = race_df['NO'][race_df['LAP']==lap].values)
    gaps_df = pd.DataFrame(distances(race_df['TIME'][race_df.index.get_level_values(1)==lap]),columns=list(race_df.index.get_level_values(0)[race_df.index.get_level_values(1)==lap].values),index = race_df.index.get_level_values(0)[race_df.index.get_level_values(1)==lap].values)
#     print(gaps_df.head(3))
    gaps_df['LAP'] = lap

    gaps_df.reset_index(level=0, inplace=True)

    gaps_df.rename({'index':'NO'},axis='columns',inplace=True)  

#     gaps_df.set_index(['NO','LAP'], inplace=True)
    
    d_gaps['gaps_df_lap_'+str(lap)] = gaps_df
    
#     gaps_df.drop(columns = ['Unnamed: 0'], inplace=True)

In [705]:
# race_df.index.get_level_values(0)[race_df.index.get_level_values(1)==36]

In [706]:
# race_df.set_index(['NO','LAP'], inplace=True)
for i in range(0,last_lap):
    lap = i+1
#     print(lap)
    gaps_df = d_gaps['gaps_df_lap_'+str(lap)]    
    gaps_df.set_index(['NO','LAP'], inplace=True)
    if lap==1:
        race_df = race_df.merge(gaps_df, on=['NO','LAP'], how="outer")
    else:
#     elif lap!=36:
        race_df.update(gaps_df, join='left')#, overwrite=True, filter_func=None, raise_conflict=False)

In [731]:
new_df = race_df.merge(pit_stop_df, on=['NO', 'LAP'], how='outer')

In [732]:
# race_df['TIME'] = race_df['TIME'].apply(lambda x: float(x.split(':')[0])*60 + float(x.split(':')[1]))

In [733]:
def pit_time(x):
    if isinstance(x,float):
        return x
    else:
        if ':' in x:
            return int(x.split(':')[0])*60 + float(x.split(':')[1])
        else:
            return float(x)

In [730]:
new_df['TOTAL TIME'][new_df['TOTAL TIME'].notna()] = new_df['TOTAL TIME'][new_df['TOTAL TIME'].notna()].apply(pit_time)

# new_df['mean_pit'] = np.mean(new_df['TOTAL TIME'][new_df['TOTAL TIME'].notna()])

# new_df['median_pit'] = np.median(new_df['TOTAL TIME'][new_df['TOTAL TIME'].notna()])

/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [558]:
new_df.to_csv('new_df.csv',index=False)
# new_df = pd.read_csv('new_df.csv')

In [736]:
new_df['TOTAL TIME']#[new_df['TOTAL TIME'].notna()].apply(pit_time)

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
5       22.087
6          NaN
7          NaN
8          NaN
9          NaN
10         NaN
11         NaN
12         NaN
13         NaN
14         NaN
15         NaN
16         NaN
17         NaN
18      23.897
19         NaN
20      32.608
21         NaN
22         NaN
23         NaN
24         NaN
25         NaN
26         NaN
27         NaN
28         NaN
29         NaN
         ...  
1425       NaN
1426       NaN
1427       NaN
1428       NaN
1429       NaN
1430       NaN
1431       NaN
1432       NaN
1433       NaN
1434       NaN
1435       NaN
1436       NaN
1437       NaN
1438       NaN
1439       NaN
1440       NaN
1441       NaN
1442       NaN
1443       NaN
1444       NaN
1445       NaN
1446       NaN
1447       NaN
1448       NaN
1449       NaN
1450       NaN
1451       NaN
1452       NaN
1453    47.632
1454    23.540
Name: TOTAL TIME, Length: 1455, dtype: float64

In [560]:
new_df.set_index(['NO','LAP'], inplace=True)

In [561]:
# # new_df['stop_count'] = np.nan
# new_df['stop_count'] = new_df['STOP']
# ###filling the first lap stop_count0s
# new_df['stop_count'][(new_df.index.get_level_values(1)==1) & (new_df['STOP']!=1)] = 0

In [562]:
new_df['d_stop'] = new_df['STOP']>0
new_df['target'] = np.nan

In [563]:
def reset_lap_counter(lap,pits):
#     print('lap',lap)
    return lap - np.append(np.intersect1d(pits,np.arange(1,lap)),0).max()

In [564]:
# # for j in new_df.index.get_level_values('NO').unique():  
# for j in new_df.NO.unique():  
# #     x = pd.concat([new_df['stop_count'][new_df.index.get_level_values('NO')==j].unstack(0).ffill().stack(0)],axis=1).swaplevel();
#     x = pd.concat([new_df['stop_count'][new_df.index.get_level_values('NO')==j].unstack(0).ffill().stack(0)],axis=1).swaplevel();
#     x.columns = ['stop_count'];
# #     new_df.update(x,join='left');
# #     new_df['target'][(new_df.index.get_level_values(0)==j)] = new_df['d_stop'][(new_df.index.get_level_values(0)==j)].shift(-1)

In [565]:
# # filling the first lap stop_count0s
# # new_df['stop_count'][(new_df.index.get_level_values(1)== 1) & (new_df['STOP'] != 1)] = 0
# # new_df['stop_count'][(new_df['LAP']== 1) & (new_df['STOP'] != 1)] = 0
# new_df['d_stop'] = new_df['STOP'] > 0

In [566]:
# _num = 44
# # new_df[new_df.index.get_level_values(0)==_num]['laps_since_pit']=new_df.index.get_level_values(1)
# new_df[new_df.NO==_num][['laps_since_pit','LAP']] #= new_df.LAP

In [567]:
new_df['laps_since_pit'] = new_df.index.get_level_values('LAP')
# new_df['laps_since_pit'] = new_df['laps_since_pit'][new_df['NO']==_num].apply(lambda x: reset_lap_counter(x,pits))
# new_df['laps_since_pit'] = new_df['laps_since_pit'][new_df.index.get_level_values(0)==_num].apply(lambda x: reset_lap_counter(x,pits))
pits = new_df.index.get_level_values('LAP')[(new_df['d_stop'])&(new_df.index.get_level_values('NO')==7)].values
# new_df['laps_since_pit'] = new_df['laps_since_pit'][new_df.index.get_level_values(0)==7].apply(lambda x: reset_lap_counter(x,pits))

In [568]:
new_df['stop_count'] = new_df['STOP']
new_df['stint'] = new_df['STOP']
new_df['stop_count'][(new_df.index.get_level_values(1)==1) & (new_df['STOP']!=1)] = 0
new_df['laps_since_pit'] = new_df.index.get_level_values('LAP')
# new_df['stop_count'][(new_df.LAP==1) & (new_df['STOP']!=1)] = 0
# for _num in new_df.NO.unique():
for _num in new_df.index.get_level_values('NO').unique():
    pits = new_df.index.get_level_values('LAP')[(new_df['d_stop'])&(new_df.index.get_level_values('NO')==_num)].values
#     print(_num,pits)
#     pits = new_df['LAP'].iloc[new_df['LAP'][(new_df['d_stop'])&(new_df['NO']==_num)].index]
    new_df['stop_count'][new_df.index.get_level_values('NO')==_num] = new_df['stop_count'][new_df.index.get_level_values('NO')==_num].ffill()
    new_df['stint'][new_df.index.get_level_values('NO')==_num] = new_df['STOP'][new_df.index.get_level_values('NO')==_num].bfill().ffill()
    new_df['laps_since_pit'][new_df.index.get_level_values(0)==_num] = new_df['laps_since_pit'][new_df.index.get_level_values(0)==_num].apply(lambda x: reset_lap_counter(x,pits))
#     new_df['stop_count'][new_df['NO']==_num] = new_df['stop_count'][new_df['NO']==_num].ffill()
#     new_df['stint'][new_df['NO']==_num] = new_df['STOP'][new_df['NO']==_num].bfill().ffill()

/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-ver

In [569]:
new_df.to_csv('new_df.csv')#,index=False)
# new_df = pd.read_csv('new_df.csv')
# new_df.set_index(['NO','LAP'], inplace=True)

In [570]:
new_df.head(3)

,,GAP,TIME,race_id,cumtime,progress,temp_lap,5,77,7,33,14,19,11,27,55,10,9,28,18,94,8,44,3,DRIVER,ENTRANT,TIME OF DAY,STOP,DURATION,TOTAL TIME,d_stop,target,laps_since_pit,stop_count,stint
NO,LAP,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,1,PIT,112.494,25.0,112.494,0.014286,1.0,0.000,-1.166,-3.120,-4.081,-5.875,-6.993,-8.639,-10.216,-11.564,-13.566,-16.128,-17.211,-18.451,-19.209,-20.329,-22.434,-23.652,S. VETTEL,Scuderia Ferrari,14:04:42,1.0,19.097,19.097,True,NaN,1,1.0,1.0
77,1,PIT,113.660,25.0,113.660,0.014286,1.0,1.166,0.000,-1.954,-2.915,-4.709,-5.827,-7.473,-9.050,-10.398,-12.400,-14.962,-16.045,-17.285,-18.043,-19.163,-21.268,-22.486,V. BOTTAS,Mercedes AMG Petronas F1 Team,14:04:43,1.0,18.717,18.717,True,NaN,1,1.0,1.0
7,1,PIT,115.614,25.0,115.614,0.014286,1.0,3.120,1.954,0.000,-0.961,-2.755,-3.873,-5.519,-7.096,-8.444,-10.446,-13.008,-14.091,-15.331,-16.089,-17.209,-19.314,-20.532,K. RAIKKONEN,Scuderia Ferrari,14:04:45,1.0,19.433,19.433,True,NaN,1,1.0,1.0
